In [5]:
# Custom image:
# Please use a public-access link, such as a google image
# http:// or https:// at start, .png or .jpg at end
image_url = 'https://prod-images.tcm.com/Master-Profile-Images/MarilynMonroe.jpg?raw=true'
# Example: 'https://prod-images.tcm.com/Master-Profile-Images/MarilynMonroe.jpg?raw=true'

# Define the spacing between dots (smaller values create a denser pattern)
# For best results, keep to (2 * num_lines) - 1 (approximately)
dot_spacing = 8
# 5 8 15

# Define number of unique dot weights:
# Also defines largest pixel radius
num_lines = 5
# 3 5 10

# Color, in hex.
color =   'ffffff'
# Red:    'ff0000'
# Green:  '00ff00'
# Blue:   '0000ff'
# White:  'ffffff'

# Name your finalized image:
# .png will be automatically added.
filename = 'abacadabra'


In [7]:
import cv2
import numpy as np
from skimage import io
import math

# Load image
image = io.imread(image_url)
image = cv2.resize(image, (800, 600))

# Convert the image from RGB to BGR format
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Dividing image
partition_levels = [int(round((256 // num_lines) * i)) for i in range(1, num_lines)]
quantized_image = np.floor_divide(image, 256 // num_lines) * (256 // num_lines)
cv2.imshow('quan', quantized_image)

# Collect the hexcode color value
color = color.lstrip('#')
r, g, b = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
fin_color = (b, g, r)

# Create a white canvas to draw dots on
height, width = image.shape
halftone = np.ones((height, width, 3), dtype=np.uint8) * 255

# Loop through the image and draw dots based on pixel intensity
for y in range(0, height, dot_spacing):
    for x in range(0, width, dot_spacing):
        intensity = image[y, x]
        dot_radius = num_lines
        for i, level in enumerate(partition_levels):
            if intensity > level:
                dot_radius = num_lines - (i + 1)
            else:
                break
        cv2.circle(halftone, (x, y), dot_radius, fin_color, -1)

# Print and kill program
cv2.imshow('halfzies', halftone)

# Invert and create transparent mask
cut_mask = cv2.inRange(halftone, (250, 250, 250), (255, 255, 255))
cut_mask = cv2.bitwise_not(cut_mask)

# Set the white areas to be fully transparent (0 alpha value)
image_with_transparency = cv2.cvtColor(halftone, cv2.COLOR_BGR2BGRA)
image_with_transparency[:, :, 3] = cut_mask

cv2.imwrite('halftone.png', image_with_transparency)

cv2.waitKey(0)

-1